<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Solved by: Madelyn Rojas

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)


In [8]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

# DETALLES



*   El crimen fue un `murder` _(asesinato)_
*   ocurrió el 15 de Enero de 2018
*   en `SQL City`

---

# DESCUBRIR INFORMACION BASICA

## A) Conocer las tablas a las que tengo acceso de información

In [14]:
%%sql
SELECT table_name
FROM information_schema.tables
ORDER BY table_name

,table_name
0,crime_scene_report
1,drivers_license
2,facebook_event_checkin
3,get_fit_now_check_in
4,get_fit_now_member
5,income
6,interview
7,person


## B) Conocer las columnas de cada tabla

### 0 Tabla crime_scene_report

In [15]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='crime_scene_report'

,column_name,data_type
0,date,INTEGER
1,type,VARCHAR
2,description,VARCHAR
3,city,VARCHAR


### 1 Tabla drivers_license

In [16]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='drivers_license'

,column_name,data_type
0,id,INTEGER
1,age,INTEGER
2,height,INTEGER
3,eye_color,VARCHAR
4,hair_color,VARCHAR
5,gender,VARCHAR
6,plate_number,VARCHAR
7,car_make,VARCHAR
8,car_model,VARCHAR


### 2 Tabla facebook_event_checkin

In [18]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='facebook_event_checkin'

,column_name,data_type
0,person_id,INTEGER
1,event_id,INTEGER
2,event_name,VARCHAR
3,date,INTEGER


### 3 Tabla get_fit_now_check_in

In [19]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='get_fit_now_check_in'

,column_name,data_type
0,membership_id,VARCHAR
1,check_in_date,INTEGER
2,check_in_time,INTEGER
3,check_out_time,INTEGER


### 4 Tabla get_fit_now_member

In [20]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='get_fit_now_member'

,column_name,data_type
0,id,VARCHAR
1,person_id,INTEGER
2,name,VARCHAR
3,membership_start_date,INTEGER
4,membership_status,VARCHAR


### 5 Tabla income

In [21]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='income'

,column_name,data_type
0,ssn,INTEGER
1,annual_income,INTEGER


### 6 Tabla interview

In [24]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='interview'

,column_name,data_type
0,person_id,INTEGER
1,transcript,VARCHAR


### 7 Tabla person

In [23]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name='person'

,column_name,data_type
0,id,INTEGER
1,name,VARCHAR
2,license_id,INTEGER
3,address_number,INTEGER
4,address_street_name,VARCHAR
5,ssn,INTEGER


### todas las tablas

In [10]:
%%sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name ASC, column_name ASC

,table_name,column_name,data_type
0,crime_scene_report,city,VARCHAR
1,crime_scene_report,date,INTEGER
2,crime_scene_report,description,VARCHAR
3,crime_scene_report,type,VARCHAR
4,drivers_license,age,INTEGER
5,drivers_license,car_make,VARCHAR
6,drivers_license,car_model,VARCHAR
7,drivers_license,eye_color,VARCHAR
8,drivers_license,gender,VARCHAR
9,drivers_license,hair_color,VARCHAR


## C) ERD DIAGRAMA DE RELACIONES DE ENTIDAD

<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>

# SOLUCION

## PASO 1 DESCUBRIR EL CRIMEN Y LOS TESTIGOS

CUAL FUE EL CRIMEN?

In [28]:
%%sql
SELECT description FROM crime_scene_report
WHERE date='20180115' AND type='murder' AND city='SQL City'

,description
0,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...


DESCUBRIR EL PRIMER TESTIGO

In [41]:
%%sql
SELECT id, name, address_street_name,
FROM person
WHERE address_street_name= 'Northwestern Dr'
AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name='Northwestern Dr')

,id,name,address_street_name
0,14887,Morty Schapiro,Northwestern Dr


DESCUBRIR EL SEGUNDO TESTIGO ID DE ANNABEL

In [33]:
%%sql
SELECT id, name, address_street_name FROM person WHERE address_street_name='Franklin Ave' AND name LIKE 'Annabel%'

,id,name,address_street_name
0,16371,Annabel Miller,Franklin Ave


## PASO 2 SABER DETALLES DE CADA TESTIMONIO

In [47]:
%%sql
SELECT p.name, i.transcript
FROM person p
JOIN interview i ON i.person_id = p.id
WHERE p.id IN (14887, 16371)

,name,transcript
0,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."
1,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."


## PASO 3 SEGUIR PISTAS DE CADA TESTIMONIO

### PISTAS DE Morty Schapiro

*   hombre
*   Get Fit Now Gym
*   membresía comenzaba con "48Z"
*   miembros de oro
*   placa que decía "H42W"

In [59]:
%%sql
SELECT m.id, m.person_id, m.name, p.license_id
from get_fit_now_member m
JOIN person p ON m.person_id=p.id
WHERE m.id LIKE '48Z%' AND m.membership_status='gold'

,id,person_id,name,license_id
0,48Z7A,28819,Joe Germuska,173289
1,48Z55,67318,Jeremy Bowers,423327


In [60]:
%%sql
SELECT id, plate_number FROM drivers_license WHERE plate_number LIKE '%H42W%'

,id,plate_number
0,183779,H42W0X
1,423327,0H42W2
2,664760,4H42WR


### PISTAS DE Annabel Miller

*   asesino en mi gimnasio
*   el 9 de enero

In [69]:
%%sql
SELECT m.name, c.check_in_date, c.check_in_time, c.check_out_time
FROM get_fit_now_member m
JOIN get_fit_now_check_in c ON m.id= c.membership_id
WHERE m.name ='Annabel Miller' AND c.check_in_date=20180109


,name,check_in_date,check_in_time,check_out_time
0,Annabel Miller,20180109,1600,1700


In [76]:
%%sql
SELECT m.id, m.name, c.check_in_date, c.check_in_time, c.check_out_time, m.membership_status
FROM get_fit_now_member m
JOIN get_fit_now_check_in c ON m.id= c.membership_id
WHERE c.check_in_date>=20180109 AND c.check_out_time > 1600 AND m.name <> 'Annabel Miller'

,id,name,check_in_date,check_in_time,check_out_time,membership_status
0,48Z7A,Joe Germuska,20180109,1600,1730,gold
1,48Z55,Jeremy Bowers,20180109,1530,1700,gold


In [88]:
%%sql
SELECT *
FROM person
WHERE name IN ('Jeremy Bowers','Joe Germuska')

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [89]:
%%sql
SELECT *
FROM facebook_event_checkin
WHERE date = 20180115 AND person_id IN (28819,67318)

,person_id,event_id,event_name,date
0,67318,4719,The Funky Grooves Tour,20180115


# VERIFICAR SI LO RESOLVISTE

In [90]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

In [91]:
%%sql
SELECT p.name, i.transcript
FROM person p
JOIN interview i ON i.person_id = p.id
WHERE p.id =67318

,name,transcript
0,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."



*   Me contrató una mujer
*   Mucho dinero
*   mide alrededor de 5'5" (65") o 5'7" (67")
*   pelo rojo
*   Tesla Model S
*   asistió al Concierto Sinfónico SQL
*   tres veces en diciembre de 2017

In [98]:
%%sql
SELECT person_id, COUNT(person_id)
FROM facebook_event_checkin
WHERE date >= 20171201 AND date <= 20171231 AND event_name LIKE 'SQL Symphony Concert'
GROUP BY person_id
HAVING COUNT(person_id)=3

,person_id,count(person_id)
0,24556,3
1,99716,3


In [107]:
%%sql
SELECT p.id, p.name, d.height, d.car_model
FROM person p
JOIN drivers_license d ON p.license_id=d.id
WHERE d.gender='female' AND hair_color='red' AND d.height >= 65 AND d.height <= 67 AND d.car_model LIKE '%Model S%' AND p.id IN (99716,24556)

,id,name,height,car_model
0,99716,Miranda Priestly,66,Model S


In [108]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True